# Packages

In [ ]:
#!/usr/bin/env python
# coding: utf-8
import requests
import numpy as np
import json
import os
import time as tm
import pandas as pd
import http.client
import io
import boto3
import zipfile
from threading import Thread
import logging
from datetime import datetime
import time
from operator import itemgetter
import xlsxwriter

# Define API Call Variables

In the following codeblock you'll need to input data for the following variables:

***app_group_api_key*** - This is the string that allows you to access the API calls for an App Group. The App Group API Key can be found in the Braze Dashboard under Settings -> Developer Console -> Rest API Keys. An App Group API key that does not have sufficient access grants may result in an error.

***API_URL*** - This is the URL used to make the Rest API Call.The current value of 'https://rest.iad-01.braze.com' may need to be updated to match the Cluster of your Braze instance. For example, your the cluster of your Braze instance may be 02, and you would update the url to 'https://rest.iad-02.braze.com'. You can find the integer value for the API URL by checking the same value next to "dashboard-0" in the URL you use to access the Braze Dashboard.

***EXPORT_DATE*** - This field is optional, only if you have run the segment export on a prior date to the same S3 bucket. It can be left blank and will export the most recent user profile data for the selected SEGMENT_ID. If not, enter a date when the export was run previously in the following format: 'YYYY-MM-DD'. All other date formats will fail to return results

***SEGMENT_ID*** - This is the Segment API Identifier used to return user data from the segment for the API call. This script can only return results for one segment at a time, and it is recommmended that the segment have no more than 200k users due to hardware limitations that were verified during testing. The Segment API Identifier can be found in the Braze Dashboard under Settings -> Developer Console -> Additional API Identifiers. Under the dropdown menu select 'Segments' and then click the 'Search for a value' dropdown to see a list of segments. Select the segement name that you wish to return results for and copy the value listed under "API Identifier".

***The App Group API Key and Segment ID should never be shared outside of your organization, or be saved in a publically accessible workspace.***

In [ ]:
app_group_api_key = 
now = datetime.now().strftime("%Y-%m-%d")
API_URL = "https://rest.iad-01.braze.com"
EXPORT_DATE = []
SEGMENT_ID =

REQUEST_HEADERS = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + app_group_api_key
}

FIELDS_TO_EXPORT = ["braze_id", "custom_attributes", "country", "total_revenue", "push_subscribe",
                    "email_subscribe", "custom_events", "purchases", "devices", "created_at", "apps",
                    "campaigns_received", "canvases_received", "cards_clicked", "push_tokens"]


# Define S3 Client Variables & Initializing the S3 Client

The codeblock below will initialize the client for Amazon S3 once the following values for the following variables have been added:

***access_key*** - Listed under "AWS Access ID"

***secret_key*** - Listed under "AWS Secret Access Key"

***region_name*** - The region that your S3 bucket is listed under

***user_export_bucket_name*** - The name of the S3 storage bucket that you would like to store the User Profile Export in. 

All of these values, with the exception of the user_export_bucket_name can be found in the Braze Dashboard under "Integrations" -> "Technology Partners" -> "AWS Cloud Storage" -> "Data Export Using AWS Credentials".

If there are no values currently listed in this section of the Braze Dashboard, you will need to work with your System Admin to either create them for the first time, or access them. In the event that you are using MFA for AWS S3, you will need to create an account that does not require the use of MFA, as otherwise the export will fail.

***This script will not function without the proper integration between Braze and Amazon S3. While it is possible to modify the script so that the files are returned to your local machine, that functionality requires additional development.*** 

*You can test your credentials by entering the proper values under 'AWS Access Key', 'AWS Secret Access Key' and 'AWS S3 Bucket Name' and then press 'Test Credentials'. If you see a success message, press save. If you do not see the success message, you'll need to work with your System Admin. to create an account and S3 bucket with the correct access controls.*

**Necessary to point out**: Keep in mind costs related to a high amount of `GET` requests for the user profiles. While these costs are minimal, S3 storage is not free, so keep that in mind before making a high volume of API requests.

Once the S3 credentials have been tested and verified via the Braze Dashboard, you should be all set to store files from the `POST` request for the [User Profiles by Semgent endpoint](https://www.braze.com/docs/api/endpoints/export/user_data/post_users_segment/).

After the variables have been entered, the S3 client will be initialized, and functions will be created so that the ZIP files returned from the API request to the S3 bucket can be processed and transformed into a pandas dataframe.

In [ ]:
access_key = 
secret_key = 
region_name = 
user_export_bucket_name = 

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=region_name
)
user_export_bucket = s3.Bucket(user_export_bucket_name)

# Segment List Endpoint

Here we'll call the [Segment List API Endpoint](https://www.braze.com/docs/api/endpoints/export/segments/get_segment/) in order to return some data needed to build the dataframe and later to return user data from that segment.

In [ ]:
page, finished = 0, False
braze_segments = []
while True:
    endpoint = f"{API_URL}/segments/list?page={page}&sort_direction=desc"
    results = requests.get(endpoint, headers=REQUEST_HEADERS).json()['segments']
    if not results:
        break
    braze_segments.extend(results)
    page += 1

braze_segments_df = pd.DataFrame(braze_segments)
braze_segments_df.columns = ['segment_id', 'segment_name',
                             'segment_analytics_tracking_enabled', 'segment_tags']

braze_segments_df = braze_segments_df[braze_segments_df['segment_id'] == SEGMENT_ID]

# Defining Functions to Process User Profiles Stored in S3

In [ ]:
def process_s3_profiles_to_dataframe(objects):
    """Build a DataFrame chunk by chunk and return it.

    Temporary function for testing efficiency of building a DataFrame as we go. 
    There are number of great hosting solutions for us but most come with memory limit. Wanted to leave this function
    for troubleshooting potential memory issues there.

    Parameters
    ----------
    objects: s3.ObjectSummary
        S3 object iterator returned from `bucket.objects.filter` method

    Returns
    -------
    pd.DataFrame
        New dataframe with exported user profile from the selected objects
    """

    frame_chunks = []
    for obj in objects:
        segment_id = obj.key.split('/')[1]
        with io.BytesIO(obj.get()["Body"].read()) as zip_bytes:
            user_chunk = process_s3_zip_object(zip_bytes, segment_id)
            frame_chunk = pd.DataFrame(user_chunk)
            frame_chunks.append(frame_chunk)
    return pd.concat(frame_chunks, ignore_index=True)


def process_s3_profiles(objects, user_data):
    """Extract and process zip user profiles obtained from user segment export.

    Parameters
    ----------
    objects : s3.ObjectSummary
        S3 object iterator returned from `bucket.objects.filter` method
    user_data : list
        Store for the extracted profile objects
    """

    for obj in objects:
        segment_id = obj.key.split('/')[1]
        with io.BytesIO(obj.get()["Body"].read()) as zip_bytes:
            user_chunk = process_s3_zip_object(zip_bytes, segment_id)
            user_data.extend(user_chunk)


def process_s3_zip_object(zip_bytes, segment_id):
    """Extract the zip file contents and process each text file within that zip file. 
    Text files extracted contain user data JSONs, separated by new line.

    Parameters
    ----------
    zip_bytes : io.BytesIO
    segment_id : string

    Returns
    -------
    list 
        Extracted user profile dictionaries from the zip file
    """

    profiles = []
    with zipfile.ZipFile(zip_bytes) as open_zip:
        for user_file in open_zip.namelist():
            with open_zip.open(user_file) as users:
                for line in users:
                    user_profile = json.loads(line.decode('utf-8'))
                    user_profile['segment_id'] = segment_id
                    profiles.append(user_profile)
    return profiles

# Define Functions for Processing Campaign Data

The below codeblock:defines functions to enable `GET` requests from the [Campaign Details Endpoint](https://www.braze.com/docs/api/endpoints/export/campaigns/get_campaign_details/) for one or many campaign_ids. It also creates functions to enable the creation of the Channel Combo and Custom Events used in Campaigns.

The URL may need to be updated in the same manner as above following 'iad-0' depending on the cluster of your Braze Instance. For example, you may need to update the string "https://rest.iad-01.braze.com/campaigns/" to "https://rest.iad-02.braze.com/campaigns/" if the Cluster for your Braze instance is 02.

The MAX_RETRIES variable is the number of times that the script will attempt to make a request to the the API Endpoint. If the number is increased the script will take longer to return results from the Campaign Details Endpoint.

In [ ]:
MAX_RETRIES=3

def process_campaign_id(campaign_id, endpoint):
    requests_made = 0
    while requests_made < MAX_RETRIES:
        try:
            response = requests.get("https://rest.iad-01.braze.com/campaigns/"+endpoint+"?campaign_id="+campaign_id, headers=REQUEST_HEADERS
            )
            return response.json()
        except requests.exceptions.HTTPError:
            requests_made += 1
            tm.sleep(0.5)
            if requests_made >= MAX_RETRIES:
                raise



### processes a range of ids
def process_campaign_id_range(campaign_id_range, endpoint, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for campaign_id in campaign_id_range:
        store[campaign_id] = process_campaign_id(campaign_id, endpoint)
    return store



def threaded_process_campaigns(nthreads, campaign_id_range, endpoint):
    """process the id range in a specified number of threads"""
    try:
        store = {}
        threads = []
        for i in range(nthreads):
            campaign_ids = campaign_id_range[i::nthreads]
            t = Thread(target=process_campaign_id_range,
                       args=(campaign_ids, endpoint, store))
            threads.append(t)

        for t in threads:
         t.start() 
        
        for t in threads:
            t.join()

        return store

    except Exception as e:
        logging.error("Threading exception: "+str(e))
        tm.sleep(30)


def get_campaign_id(df_column):
    try:
        return df_column.get('api_campaign_id')
    except AttributeError:
        return float('NaN')


def get_message_variation_id(df_column):
    try:
        return df_column.get('variation_api_id')
    except AttributeError:
        return float('NaN')


def parse_channel(row):
    if row.num_channels > 0:
        return row.channel
    elif type(row.campaigns_received) != dict:
        return "No Messages Received"
    else:
        return "Unable to Retrieve Campaign Details"

def parse_channel_combo(row):
    if type(row.channel_combo) != float:
        return row.channel_combo
    elif row.channel == "No Messages Received":
        return "No Messages Received"
    else:
        return "Unable to Retrieve Campaign Details"


def get_campaign_custom_event(df_column):
    try:
        return df_column.get('custom_event_name')
    except AttributeError:
        return float('NaN')

# Define Field Getters to Enable Segment Analytics

The functions defined in the codeblocks below will get the corresponding fields from nested dictionaries stored in dataframes columns that are returned from the User Profiles Endpoint.

In [ ]:
def get_email_open_engagement(df_column):
    try:
        return df_column.get('opened_email')
    except AttributeError:
        return False


def get_email_click_engagement(df_column):
    try:
        return df_column.get('clicked_email')
    except AttributeError:
        return False


def get_push_engagement(df_column):
    try:
        return df_column.get('opened_push')
    except AttributeError:
        return False


def get_iam_engagement(df_column):
    try:
        return df_column.get('clicked_in_app_message')
    except AttributeError:
        return False


def get_conversions(df_column):
    try:
        return df_column.get('converted')
    except AttributeError:
        return False

### Create get engagement


def calc_engagement(series):
    return series.sum()/series.count()


def get_cards_clicked(row):
    if row.channel == 'No Messages Received':
        return 0
    else:
        return len(row.cards_clicked)

In [ ]:
def days_between(d1, d2):
    d1 = datetime.datetime.strptime(str(d1), '%Y-%m-%dT%H:%M:%S.%f%z')
    d2 = datetime.datetime.strptime(str(d2), '%Y-%m-%dT%H:%M:%S.%f%z')
    return (d2 - d1).days


def get_custom_event_name(df_column):
    try:
        return df_column.get('name')
    except AttributeError:
        return float('NaN')


def get_custom_event_count(df_column):
    try:
        return df_column.get('count')
    except AttributeError:
        return float('NaN')


def get_custom_event_first_date(df_column):
    try:
        return df_column.get('first')
    except AttributeError:
        return float('NaN')


def get_custom_event_last_date(df_column):
    try:
        return df_column.get('last')
    except AttributeError:
        return float('NaN')

In [ ]:
def get_notifications_enabled(df_column):
    try:
        return df_column.get('notifications_enabled')
    except AttributeError:
        return False


def get_token(df_column):
    try:
        return df_column.get('token')
    except AttributeError:
        return 'None'


def get_platform(df_column):
    try:
        return df_column.get('platform')
    except AttributeError:
        return 'No platform token'

# Export Data from S3 for either Today or a Prior Date

The below codeblock will do the following depending on the value entered above for the EXPORT_DATE variable:

***If the EXPORT_DATE is left blank***:

- Make a request to the [Users by Segment Endpoint](https://www.braze.com/docs/api/endpoints/export/user_data/post_users_segment/)

- Process the user profile data returned to S3 following the successful request for the selected SEGMENT_ID

- Displays the number of user profiles that have been returned.

If the value returned is 0 it is likely that some of the above variables were not configured properly. You'll need to double check and try again.

If the number of user profiles exported is low, it could be because of latency between the Braze API and S3. Try running the code block again and see if the number of users returned increases

***If the EXPORT_DATE is a properly formatted data from a prior export***

- Process the user profile data returned to S3 following the successful request for the selected SEGMENT_ID

- Displays the number of user profiles that have been returned.

If the EXPORT_DATE is not formatted 'YYYY-MM-DD' the below codeblock will fail and you will be asked to try again.

If completed successfully, the segment_df data should return a dataframe for all user profiles from the segment, along with data from the fields listed out in the *FIELDS_TO_EXPORT* variable. Each row in the dataframe corresponds to one user profile within the selected segment.

In [ ]:
if len(EXPORT_DATE) == 0:
    object_prefix_by_segment_id = []
    payload = {
        "segment_id": SEGMENT_ID,
        "fields_to_export": FIELDS_TO_EXPORT
    }
    res = requests.post(f"{API_URL}/users/export/segment",
                        headers=REQUEST_HEADERS, json=payload)
    res_data = res.json()
    print(res_data)
    EXPORT_DATE = datetime.today().strftime('%Y-%m-%d')
    objects = user_export_bucket.objects.filter(
        Prefix=f"segment-export/{SEGMENT_ID}/{EXPORT_DATE}")
    tm.sleep(300)
    print("Waiting for data to be returned from the Users by Segment Endpoint.")
    start = time.time()
    user_data = []
    print("Reading exported user data from S3")
    process_s3_profiles(objects, user_data)
    print(f"Took {(time.time() - start):.2f}s")
    print(len(user_data))
elif len(EXPORT_DATE) == 10 and EXPORT_DATE.count('-') == 2 and len(EXPORT_DATE) > 0:
    year, month, day = EXPORT_DATE.split('-')
    isValidDate = True
    try:
        datetime(int(year), int(month), int(day))
    except ValueError:
        print("Input date is not the valid YYYY-MM-DD format. Please return to the Define Variables cell and try again enter a properly formatted Date.")
        isValidDate = False
    if(isValidDate):
        objects = user_export_bucket.objects.filter(
            Prefix=f"segment-export/{SEGMENT_ID}/{EXPORT_DATE}")
        start = time.time()
        user_data = []
        print("Reading exported user data from S3")
        process_s3_profiles(objects, user_data)
        print(f"Took {(time.time() - start):.2f}s")
        print(len(user_data))
else:
    print("This is the text that will display if export date is neither blank nor properly formatted.")

In [ ]:
segment_df_raw = pd.DataFrame(user_data)\
    .dropna(subset=['braze_id'])


segment_df = pd.merge(segment_df_raw, braze_segments_df,
                      how='left',
                      left_on=['segment_id'],
                      right_on=['segment_id'],
                      suffixes=['_from_user_segment_endpoint', '_from_segment_list'])

# Creating Separate Dataframes for Each KPI

The below codeblock will split the segment_df into the appropriate dataframes so that the following analytical outputs can be viewed from the selected Segment:

1. Rolling Retention
2. Purchasing Rates & 
3. Purchase Retention
4. Session Engagement Metrics
5. Custom Event Metrics
5. Message Engagement Rates
6. Custom Events used in Campaigns
7. Opt-In Rates for Push and Email

1-5 will also be crossed by the following dimensions from Message Engagement so that the impact of different messaging strategies can be viewed at the segement level:

- Channel
- Channel Combo
- Campaign Tag

In the event that a segment fails one of the checks below, you can skip those sections in the script. For example, say you are tracking session data, but not purchasing data. Skip the purchasing codeblocks and comment out the final outputs associated with those metrics.

In [ ]:
rolling_retention_columns = ['braze_id', 'segment_id',
                             'apps', 'segment_name', 'segment_tags']
purchasing_stats_columns = ['braze_id',
                            'segment_id', 'apps', 'segment_name', 'segment_tags', 'purchases', 'total_revenue']
sessions_stats_columns = ['braze_id', 'segment_id',
                          'apps', 'segment_name', 'segment_tags']
custom_events_stats_columns = ['braze_id', 'segment_id', 'apps',
                               'segment_name', 'segment_tags', 'custom_events']
engagement_stats_columns_all = ['braze_id', 'segment_id', 'country', 'apps', 'segment_name',
                                'segment_tags', 'campaigns_received', 'canvases_received', 'cards_clicked']
engagement_stats_columns_canvas = ['braze_id', 'segment_id', 'country', 'apps',
                                   'segment_name', 'segment_tags', 'canvases_received', 'cards_clicked']
engagement_stats_columns_campaigns = ['braze_id', 'segment_id', 'country', 'apps',
                                      'segment_name', 'segment_tags', 'campaigns_received', 'cards_clicked']
opt_ins_columns_all = ['braze_id', 'segment_id', 'segment_name', 'apps', 'push_tokens', 'email_subscribe', 'email_opted_in_at', 'push_subscribe',
                       'push_opted_in_at', 'email_unsubscribed_at', 'push_unsubscribed_at']
opt_ins_columns_email = ['braze_id', 'segment_id', 'segment_name', 'apps', 'push_tokens',
                         'email_subscribe', 'email_opted_in_at', 'email_unsubscribed_at']
opt_ins_columns_push = ['braze_id', 'segment_id', 'segment_name', 'apps', 'push_tokens',
                        'push_subscribe', 'push_opted_in_at', 'push_unsubscribed_at']

users_have_sessions = "apps" in segment_df
users_have_purchases = "purchases" in segment_df
users_have_custom_events = "custom_events" in segment_df
users_received_campaigns = "campaigns_received" in segment_df
users_received_canvas = "canvases_received" in segment_df
users_subscribed_email = "email_subscribe" in segment_df
users_subscribed_push = "push_subscribe" in segment_df


if users_have_sessions == True:
    segment_rolling_retention_pre_apps = segment_df[rolling_retention_columns]
    segment_rolling_retention_pre_apps = segment_rolling_retention_pre_apps.reset_index()
else:
    print("Users in these Segments do not have Retention Data")

if users_have_purchases == True:
    segment_purchasing_stats_pre_apps = segment_df[purchasing_stats_columns]
    segment_purchasing_stats_pre_apps = segment_purchasing_stats_pre_apps.reset_index()
else:
    print("Users in these Segments do not have Purchasing Data")

if users_have_sessions == True:
    segment_sessions_stats_pre_apps = segment_df[sessions_stats_columns]
    segment_sessions_stats_pre_apps = segment_sessions_stats_pre_apps.reset_index()
else:
    print("Users in these Segments do not have Session Data")

if users_have_custom_events == True:
    segment_custom_event_stats_pre_custom_event = segment_df[custom_events_stats_columns]
    segment_custom_event_stats_pre_custom_event = segment_custom_event_stats_pre_custom_event.reset_index()
else:
    print("Users in these Segments do not have Custom Event Data")

if (users_received_campaigns == True and users_received_canvas == True):
    segment_engagement_stats_pre_apps = segment_df[engagement_stats_columns_all]
elif (users_received_campaigns == False and users_received_canvas == True):
    segment_engagement_stats_pre_apps = segment_df[engagement_stats_columns_canvas]
elif (users_received_campaigns == True and users_received_canvas == False):
    segment_engagement_stats_pre_apps = segment_df[engagement_stats_columns_campaigns]
elif (users_received_campaigns == False and users_received_canvas == False):
    print("Users in these Segments do not have Engagement Data")

if (users_subscribed_email == True and users_subscribed_push == True):
    segment_opt_in_stats_pre_apps = segment_df[opt_ins_columns_all]
elif (users_subscribed_email == False and users_subscribed_push == True):
    segment_opt_in_stats_pre_apps = segment_df[users_subscribed_push]
elif (users_subscribed_email == True and users_subscribed_push == False):
    segment_opt_in_stats_pre_apps = segment_df[users_subscribed_email]
elif (users_subscribed_email == False and users_subscribed_push == False):
    print("Users in these Segments do not have Opt-In Data")

# Campaign & Engagement Data

The below codeblocks will complete the following tasks:

- Return all of the campaign ids received by the exported Segment

- Send `GET` results from the [Campaign Details API](https://www.braze.com/docs/api/endpoints/export/campaigns/get_campaign_details/#campaign-details-endpoint-api-response) and process the data that is returned.

    - Users that received messages from campaign_ids that do not have details returned will be assigned the 'Unable to Retrieve Campaign Details' value for both Channel and Channel Combo.

- Create the Channel_Combo dimension. Please note that the Channel Combo is being created at the Campaign Level and not the User Level. 

- Removing Users in the control_group for multivariate campaigns

- Cleaning the Channel names and Channel Combo names

- Creating the dataframe used to caclulate Message Engagement Metrics

- Creating dataframes used to cross other metrics with Channel, Channel Combo, and Campaign Tag


In [ ]:
segment_engagement_temp = segment_engagement_stats_pre_apps.explode(
    'campaigns_received')
segment_engagement_temp['campaign_id'] = list(
    map(get_campaign_id, segment_engagement_temp['campaigns_received']))

braze_campaigns = segment_engagement_temp[segment_engagement_temp['campaign_id'].isnull(
) == False]['campaign_id']
braze_campaigns = list(set(braze_campaigns))

In [ ]:
campaign_dict = threaded_process_campaigns(
    10, braze_campaigns, 'details')

In [ ]:
campaign_details_df = pd.DataFrame.from_dict(campaign_dict, orient='index')
campaign_details_df = campaign_details_df.reset_index()
campaign_details_df.rename(columns={"index": "campaign_id"},
                           inplace=True)
campaign_details_df = campaign_details_df[campaign_details_df['message'] == 'success']
campaign_details_df['num_channels'] = campaign_details_df.channels.apply(len)
campaign_details_df = campaign_details_df[campaign_details_df['num_channels'] > 0]

In [ ]:
joined_campaign = pd.merge(segment_engagement_temp, campaign_details_df,
                           how='left',
                           left_on=['campaign_id'],
                           right_on=['campaign_id'],
                           suffixes=['_from_segments', '_from_campaigns'])
segment_data_engagement_stats_temp = joined_campaign
segment_data_engagement_stats_temp.rename(columns={"channels": "channel"},
                                          inplace=True)
segment_data_engagement_stats_temp['in_control']=segment_data_engagement_stats_temp.campaigns_received.apply(
    lambda x: x.get('in_control') if type(x) != float else x)
segment_data_engagement_stats_temp=segment_data_engagement_stats_temp[segment_data_engagement_stats_temp['in_control']!=True]

segment_data_engagement_stats_temp.loc[:, 'channel'] = segment_data_engagement_stats_temp.apply(
    parse_channel, axis=1)
segment_data_engagement_stats_temp = segment_data_engagement_stats_temp.explode(
    'channel')

In [ ]:
segment_data_engagement_stats_temp['channel'] = segment_data_engagement_stats_temp.channel.apply(
    lambda x: 'mobile_push' if x == 'android_push' or x == 'ios_push' else x)
segment_data_engagement_stats_temp['channel'] = segment_data_engagement_stats_temp.channel.apply(
    lambda x: 'in_app_message' if x == 'legacy_in_app_message' or x == 'trigger_in_app_message ' or x == 'trigger_in_app_message' else x)
segment_data_engagement_stats_temp['channel'] = segment_data_engagement_stats_temp.channel.apply(
    lambda x: x.replace("_", " "))
segment_data_engagement_stats_temp['channel'] = segment_data_engagement_stats_temp.channel.apply(
    lambda x: x.title())

In [ ]:
segment_data_channel_combo = segment_data_engagement_stats_temp[(segment_data_engagement_stats_temp['channel'] != 'No Messages Received')]
segment_data_channel_combo = segment_data_engagement_stats_temp[segment_data_engagement_stats_temp['channel'] != 'Unable To Retrieve Campaign Details']
segment_data_channel_combo = segment_data_channel_combo[[
    'braze_id', 'channel']].drop_duplicates()
segment_data_channel_combo = segment_data_channel_combo.dropna(subset=[
                                                               'channel'])
segment_data_channel_combo = segment_data_channel_combo.groupby('braze_id')
segment_data_channel_combo = segment_data_channel_combo.apply(
    lambda x: x['channel'].unique()).reset_index()
segment_data_channel_combo.columns = ['braze_id', 'channel_combo']
segment_data_channel_combo['channel_combo'] = segment_data_channel_combo.channel_combo.apply(
    lambda x: np.ndarray.tolist(x))
segment_data_channel_combo['channel_combo'] = segment_data_channel_combo['channel_combo'].apply(
    lambda x: list(set(x)))
segment_data_channel_combo['channel_combo'] = segment_data_channel_combo.channel_combo.apply(
    sorted)
segment_data_channel_combo['channel_combo'] = [
    ', '.join(map(str, l)) for l in segment_data_channel_combo['channel_combo']]
segment_data_channel_combo = segment_data_channel_combo.drop_duplicates()

In [ ]:
segment_data_engagement_stats = pd.merge(segment_data_engagement_stats_temp, segment_data_channel_combo,
                                         how='left',
                                         left_on=['braze_id'],
                                         right_on=['braze_id'],
                                         suffixes=['_from_engagement', '_from_channel_combo'])

In [ ]:
segment_data_engagement_stats.loc[:, 'channel_combo'] = segment_data_engagement_stats.apply(
    parse_channel_combo, axis=1)

users_per_channel_df = segment_data_engagement_stats.groupby(
    ['segment_name', 'segment_id','channel']).agg(num_users=('braze_id', 'nunique'))
users_per_channel_df = users_per_channel_df.reset_index(level=[0, 1, 2])
users_per_channel_combo_df = segment_data_engagement_stats.groupby(
    ['segment_name', 'segment_id','channel_combo']).agg(num_users=('braze_id', 'nunique'))
users_per_channel_combo_df = users_per_channel_combo_df.reset_index(level=[
                                                                    0, 1, 2])
users_per_campaign_tags_df = segment_data_engagement_stats.explode('tags')
users_per_campaign_tags_df['tags'] = users_per_campaign_tags_df.tags.fillna(
    'No Messages')
users_per_campaign_tags_df = users_per_campaign_tags_df.groupby(
    ['segment_name', 'segment_id','tags']).agg(num_users=('braze_id', 'nunique'))
users_per_campaign_tags_df = users_per_campaign_tags_df.reset_index(level=[
                                                                    0, 1, 2])

# Calculate Engagement

The below codeblocks will return Messge Engagement rates for all channels. If the segment did not receive a channel it will simply return a value of zero under the engagement metric.

The following Message Engagement Rates will be returned:

- Number of Users

- Email Open Rate

- Email Click Rate

- Push Open Rate

- In-App Message Click Rate

- Message Conversion Rates (of all Conversion Criteria)

- Content Card Click Rate

Message Engagement Rates will be returned by:

- Segment

- Channel

- Channel Combo

- Campaign Tag

In [ ]:
segment_data_engagement_stats['campaign_engaged'] = segment_data_engagement_stats.campaigns_received.apply(
    lambda x: x.get('engaged') if type(x) != float else x)
segment_data_engagement_stats['opened_email'] = list(
    map(get_email_open_engagement, segment_data_engagement_stats['campaign_engaged']))
segment_data_engagement_stats['clicked_email'] = list(map(
    get_email_click_engagement, segment_data_engagement_stats['campaign_engaged']))
segment_data_engagement_stats['opened_push'] = list(
    map(get_push_engagement, segment_data_engagement_stats['campaign_engaged']))
segment_data_engagement_stats['clicked_iam'] = list(
    map(get_iam_engagement, segment_data_engagement_stats['campaign_engaged']))
segment_data_engagement_stats['converted'] = list(
    map(get_conversions, segment_data_engagement_stats['campaigns_received']))
segment_data_engagement_stats['converted'] = segment_data_engagement_stats.converted.fillna(
    value=False)
segment_data_engagement_stats['cards_clicked'] = segment_data_engagement_stats.cards_clicked.fillna(
    value='')
segment_data_engagement_stats.loc[:, 'cards_clicked'] = segment_data_engagement_stats.apply(
    get_cards_clicked, axis=1)

In [ ]:
engagement_by_segment_preagg = segment_data_engagement_stats.groupby(
    ['segment_name', 'segment_id'])
engagement_by_segment = engagement_by_segment_preagg.agg(
    num_users=('braze_id', 'nunique'), email_open_rate=('opened_email', calc_engagement), email_click_rate=('clicked_email', calc_engagement),
    push_open_rate=('opened_push', calc_engagement), iam_click_rate=('clicked_iam', calc_engagement), conversion_rate=('converted', calc_engagement), content_card_click_rate=('cards_clicked', calc_engagement))

In [ ]:
engagement_by_segment_and_channel_preagg = segment_data_engagement_stats.groupby(
    ['segment_name', 'segment_id', 'channel'])
engagement_by_segment_and_channel = engagement_by_segment_and_channel_preagg.agg(
    num_users=('braze_id', 'nunique'), email_open_rate=('opened_email', calc_engagement), email_click_rate=('clicked_email', calc_engagement),
    push_open_rate=('opened_push', calc_engagement), iam_click_rate=('clicked_iam', calc_engagement), conversion_rate=('converted', calc_engagement), content_card_click_rate=('cards_clicked', calc_engagement))

In [ ]:
engagement_by_segment_and_channel_combo_preagg = segment_data_engagement_stats.groupby(
    ['segment_name', 'segment_id', 'channel_combo'])
engagement_by_segment_and_channel_combo = engagement_by_segment_and_channel_combo_preagg.agg(
    num_users=('braze_id', 'nunique'), email_open_rate=('opened_email', calc_engagement), email_click_rate=('clicked_email', calc_engagement),
    push_open_rate=('opened_push', calc_engagement), iam_click_rate=('clicked_iam', calc_engagement), conversion_rate=('converted', calc_engagement), content_card_click_rate=('cards_clicked', calc_engagement))
engagement_by_segment_and_channel_combo = engagement_by_segment_and_channel_combo

In [ ]:
segment_data_engagement_stats_by_campaign_tags = segment_data_engagement_stats.explode(
    'tags')
engagement_by_segment_and_campaign_tag_preagg = segment_data_engagement_stats_by_campaign_tags.groupby([
                                                                                                       'segment_name', 'segment_id', 'tags'])
engagement_by_segment_and_campaign_tag = engagement_by_segment_and_campaign_tag_preagg.agg(
    num_users=('braze_id', 'nunique'), email_open_rate=('opened_email', calc_engagement), email_click_rate=('clicked_email', calc_engagement),
    push_open_rate=('opened_push', calc_engagement), iam_click_rate=('clicked_iam', calc_engagement), conversion_rate=('converted', calc_engagement), content_card_click_rate=('cards_clicked', calc_engagement))

# Rolling Retention

The below codeblocks will return Rolling Retetion Rates. You can view the Rolling Retention Methodology [here](https://www.braze.com/resources/articles/calculate-retention-rate). 

Rolling Retention Rates will be returned by:

- Segment

- Channel

- Channel Combo

- Campaign Tag

In [ ]:
segment_rolling_retention_temp = segment_rolling_retention_pre_apps.explode(
    'apps')
segment_rolling_retention_temp = segment_rolling_retention_temp.dropna(subset=[
                                                                       'apps'])

In [ ]:
segment_rolling_retention_temp['first_used'] = segment_rolling_retention_temp['apps'].apply(
    lambda x: x.get('first_used'))
segment_rolling_retention_temp['last_used'] = segment_rolling_retention_temp['apps'].apply(
    lambda x: x.get('last_used'))
segment_rolling_retention_temp['platform'] = segment_rolling_retention_temp['apps'].apply(
    lambda x: x.get('platform'))

In [ ]:
segment_rolling_retention_temp[['first_used', 'last_used']] = segment_rolling_retention_temp[[
    'first_used', 'last_used']].apply(pd.to_datetime)
segment_rolling_retention_temp['day_num'] = (
    segment_rolling_retention_temp['last_used'] - segment_rolling_retention_temp['first_used']).dt.days

In [ ]:
segment_rolling_retention_temp['day_num'] = segment_rolling_retention_temp['day_num'].astype(
    'int')
segment_rolling_retention_raw = pd.pivot_table(segment_rolling_retention_temp,
                                               values=("braze_id"),
                                               index=("segment_name", 'segment_id',
                                                      "platform"),
                                               columns="day_num",
                                               aggfunc='nunique')\
    .fillna(0)

segment_rolling_retention_raw = segment_rolling_retention_raw[segment_rolling_retention_raw
                                                              .columns[::-1]].cumsum(axis=1)
segment_rolling_retention_raw = segment_rolling_retention_raw[
    segment_rolling_retention_raw.columns[::-1]]
segment_rolling_retention_raw["num_users"] = segment_rolling_retention_raw[0]
segment_rolling_retention_raw = segment_rolling_retention_raw.groupby(
    ['segment_name', 'segment_id', 'platform']).sum()
segment_rolling_retention = pd.concat([segment_rolling_retention_raw["num_users"],
                                       segment_rolling_retention_raw
                                       .drop(["num_users"], axis=1)
                                       .div(segment_rolling_retention_raw["num_users"], axis=0)],
                                      axis=1).fillna(0)

segment_rolling_retention=segment_rolling_retention.drop(0,axis=1)

In [ ]:
segment_engagement_user_data = segment_data_engagement_stats[[
    'braze_id', 'segment_id', 'segment_name', 'apps', 'channel', 'tags', 'channel_combo']]
segment_engagement_data_for_retention = segment_engagement_user_data.explode(
    'apps')
segment_engagement_data_for_retention = segment_engagement_data_for_retention.dropna(subset=[
                                                                                     'apps'])
segment_engagement_data_for_retention['platform'] = segment_engagement_data_for_retention['apps'].apply(
    lambda x: x.get('platform'))

In [ ]:
segment_rolling_retention_by_engagement_temp = pd.merge(segment_rolling_retention_temp.reset_index(), segment_engagement_data_for_retention.reset_index(),
                                                        how='left',
                                                        left_on=[
                                                            'braze_id', 'platform', 'segment_id', 'segment_name'],
                                                        right_on=[
                                                            'braze_id', 'platform', 'segment_id', 'segment_name'],
                                                        suffixes=['_from_retention', '_from_engagement'])

In [ ]:
segment_rolling_retention_by_engagement_temp['day_num'] = segment_rolling_retention_by_engagement_temp['day_num'].astype(
    'int')
segment_rolling_retention_by_engagement_raw = pd.pivot_table(segment_rolling_retention_by_engagement_temp,
                                                             values=(
                                                                 "braze_id"),
                                                             index=(
                                                                 "segment_name", "segment_id", "platform", "channel"),
                                                             columns="day_num",
                                                             aggfunc='nunique')\
    .fillna(0)

segment_rolling_retention_by_engagement_raw = segment_rolling_retention_by_engagement_raw[segment_rolling_retention_by_engagement_raw
                                                                                          .columns[::-1]].cumsum(axis=1)
segment_rolling_retention_by_engagement_raw = segment_rolling_retention_by_engagement_raw[
    segment_rolling_retention_by_engagement_raw.columns[::-1]]
segment_rolling_retention_by_engagement_raw["num_users"] = segment_rolling_retention_by_engagement_raw[0]
segment_rolling_retention_by_engagement_raw = segment_rolling_retention_by_engagement_raw.groupby(
    ['segment_name', 'segment_id', 'platform', "channel"]).sum()
segment_rolling_retention_by_engagement = pd.concat([segment_rolling_retention_by_engagement_raw["num_users"],
                                                     segment_rolling_retention_by_engagement_raw
                                                     .drop(["num_users"], axis=1)
                                                     .div(segment_rolling_retention_by_engagement_raw["num_users"], axis=0)],
                                                    axis=1).fillna(0)

segment_rolling_retention_by_engagement=segment_rolling_retention_by_engagement.drop(0,axis=1)

In [ ]:
segment_rolling_retention_by_engagement_temp['day_num'] = segment_rolling_retention_by_engagement_temp['day_num'].astype(
    'int')
segment_campaign_tag_data_for_retention_temp = segment_rolling_retention_by_engagement_temp.explode(
    'tags')
segment_campaign_tag_data_for_retention_temp = segment_campaign_tag_data_for_retention_temp.dropna(subset=[
                                                                                                   'tags'])
segment_rolling_retention_by_campaign_tag_raw = pd.pivot_table(segment_campaign_tag_data_for_retention_temp,
                                                               values=(
                                                                   "braze_id"),
                                                               index=(
                                                                   "segment_name", "segment_id", "platform", "tags"),
                                                               columns="day_num",
                                                               aggfunc='nunique')\
    .fillna(0)

segment_rolling_retention_by_campaign_tag_raw = segment_rolling_retention_by_campaign_tag_raw[segment_rolling_retention_by_campaign_tag_raw
                                                                                              .columns[::-1]].cumsum(axis=1)
segment_rolling_retention_by_campaign_tag_raw = segment_rolling_retention_by_campaign_tag_raw[
    segment_rolling_retention_by_campaign_tag_raw.columns[::-1]]
segment_rolling_retention_by_campaign_tag_raw["num_users"] = segment_rolling_retention_by_campaign_tag_raw[0]
segment_rolling_retention_by_campaign_tag_raw = segment_rolling_retention_by_campaign_tag_raw.groupby(
    ['segment_name', 'segment_id', 'platform', "tags"]).sum()
segment_rolling_retention_by_campaign_tag = pd.concat([segment_rolling_retention_by_campaign_tag_raw["num_users"],
                                                       segment_rolling_retention_by_campaign_tag_raw
                                                       .drop(["num_users"], axis=1)
                                                       .div(segment_rolling_retention_by_campaign_tag_raw["num_users"], axis=0)],
                                                      axis=1).fillna(0)
segment_rolling_retention_by_campaign_tag =segment_rolling_retention_by_campaign_tag.drop(0,axis=1)


In [ ]:
segment_rolling_retention_by_engagement_temp['day_num'] = segment_rolling_retention_by_engagement_temp['day_num'].astype(
    'int')
segment_rolling_retention_by_channel_combo_raw = pd.pivot_table(segment_rolling_retention_by_engagement_temp,
                                                                values=(
                                                                    "braze_id"),
                                                                index=(
                                                                    "segment_name", "segment_id", "platform", "channel_combo"),
                                                                columns="day_num",
                                                                aggfunc='nunique')\
    .fillna(0)

segment_rolling_retention_by_channel_combo_raw = segment_rolling_retention_by_channel_combo_raw[segment_rolling_retention_by_channel_combo_raw
                                                                                                .columns[::-1]].cumsum(axis=1)
segment_rolling_retention_by_channel_combo_raw = segment_rolling_retention_by_channel_combo_raw[
    segment_rolling_retention_by_channel_combo_raw.columns[::-1]]
segment_rolling_retention_by_channel_combo_raw["num_users"] = segment_rolling_retention_by_channel_combo_raw[0]
segment_rolling_retention_by_channel_combo_raw = segment_rolling_retention_by_channel_combo_raw.groupby(
    ['segment_name', 'segment_id', 'platform', "channel_combo"]).sum()
segment_rolling_retention_by_channel_combo = pd.concat([segment_rolling_retention_by_channel_combo_raw["num_users"],
                                                        segment_rolling_retention_by_channel_combo_raw
                                                        .drop(["num_users"], axis=1)
                                                        .div(segment_rolling_retention_by_channel_combo_raw["num_users"], axis=0)],
                                                       axis=1).fillna(0)

segment_rolling_retention_by_channel_combo=segment_rolling_retention_by_channel_combo.drop(0,axis=1)



# Purchasing Stats

The following purchasing metrics will be returned in the first purchasing stats dataframe:

- Number of Buyers

- Number of Repeat Buyers

- % Buyers

- % Repeat Buyers

- Number of Purchases

- Total Revenue

- Average Revenue per Buyer

- Average time to Purchase

- Purchases per Buyer

The second purchasing stats dataframe will return purchase retention rates.

Both purchasing stats dataframes will returned by:

- Segment

- Channel

- Channel Combo

- Campaign Tag

In [ ]:
num_users = segment_df.braze_id.nunique()
segment_purchasing_stats_temp = segment_purchasing_stats_pre_apps.dropna(
    subset=['apps', 'purchases'])
segment_purchasing_dates = segment_purchasing_stats_pre_apps.dropna(
    subset=['apps', 'purchases'])
segment_purchasing_dates = segment_purchasing_dates.explode(
    'purchases')
segment_purchasing_dates = segment_purchasing_dates.explode(
    'apps')

segment_purchasing_stats_temp['num_purchases'] = segment_purchasing_stats_temp['purchases'].apply(
    lambda x: sum(map(itemgetter('count'), x)))
segment_purchasing_dates['first_purchase'] = segment_purchasing_dates['purchases'].apply(
    lambda x: x.get('first'))
segment_purchasing_dates['last_purchase'] = segment_purchasing_dates['purchases'].apply(
    lambda x: x.get('last'))
segment_purchasing_dates['first_session'] = segment_purchasing_dates['apps'].apply(
    lambda x: x.get('first_used'))
segment_purchasing_dates['first_purchase'] = pd.to_datetime(
    segment_purchasing_dates['first_purchase'])
segment_purchasing_dates['last_purchase'] = pd.to_datetime(
    segment_purchasing_dates['last_purchase'])
segment_purchasing_dates['first_session'] = pd.to_datetime(
    segment_purchasing_dates['first_session'])


segment_purchasing_dates_temp = segment_purchasing_dates.groupby(['segment_name', 'segment_id', 'braze_id']).agg(first_purchase_date=(
    'first_purchase', 'min'), last_purchase_date=('last_purchase', 'max'), first_session_date=('first_session', 'min'))
segment_purchasing_dates_temp = segment_purchasing_dates_temp.reset_index(level=[
                                                                          0, 1, 2])
segment_purchasing_stats_temp = pd.merge(segment_purchasing_stats_temp, segment_purchasing_dates_temp,
                                         how='left',
                                         left_on=[
                                             'braze_id', 'segment_id', 'segment_name'],
                                         right_on=[
                                             'braze_id', 'segment_id', 'segment_name'])

segment_purchasing_stats_temp['repeat_buyer'] = segment_purchasing_stats_temp[
    'first_purchase_date'] != segment_purchasing_stats_temp['last_purchase_date']
segment_purchasing_stats_temp['repeat_buyer_id'] = segment_purchasing_stats_temp.apply(
    lambda row: row.braze_id if row.repeat_buyer == True else 'NaN', axis=1)
segment_purchasing_stats_temp['days_to_purchase'] = (
    segment_purchasing_stats_temp['first_purchase_date'] - segment_purchasing_stats_temp['first_session_date']).dt.seconds
segment_purchasing_stats_temp['days_to_purchase'] = segment_purchasing_stats_temp['days_to_purchase']/86400
segment_purchase_retention_temp = segment_purchasing_stats_temp
segment_purchase_data = segment_purchasing_stats_temp

segment_purchasing_stats_temp = segment_purchasing_stats_temp.groupby(['segment_name', 'segment_id']).agg(buyers=('braze_id', 'nunique'), repeat_buyers=('repeat_buyer_id', 'nunique'), num_purchases=(
    'num_purchases', 'sum'), total_revenue=('total_revenue', 'sum'), avg_revenue_per_buyer=('total_revenue', 'mean'), avg_time_to_purchase=('days_to_purchase', 'mean'))
segment_purchasing_stats_temp['pct_repeat_buyers'] = round(
    segment_purchasing_stats_temp.repeat_buyers/segment_purchasing_stats_temp.buyers, 2)
segment_purchasing_stats_temp['purchases_per_buyer'] = round(
    segment_purchasing_stats_temp.num_purchases/segment_purchasing_stats_temp.buyers, 2)
segment_purchasing_stats_temp['revenue_per_item_purchased'] = round(
    segment_purchasing_stats_temp.total_revenue/segment_purchasing_stats_temp.num_purchases, 2)
segment_purchasing_stats_temp['purchases_per_user'] = round(
    segment_purchasing_stats_temp.num_purchases/num_users, 2)
segment_purchasing_stats_temp['pct_buyer'] = round(
    segment_purchasing_stats_temp.buyers/num_users, 2)

segment_purchasing_stats = segment_purchasing_stats_temp

In [ ]:
segment_purchase_retention_temp['day_num'] = (
    segment_purchase_retention_temp['last_purchase_date'] - segment_purchase_retention_temp['first_purchase_date']).dt.days

segment_purchase_retention_temp['day_num'] = segment_purchase_retention_temp['day_num'].astype(
    'int')
segment_purchase_retention_raw = pd.pivot_table(segment_purchase_retention_temp,
                                                values=("braze_id"),
                                                index=("segment_name",
                                                       "segment_id"),
                                                columns="day_num",
                                                aggfunc='nunique')\
    .fillna(0)


segment_purchase_retention_raw = segment_purchase_retention_raw[segment_purchase_retention_raw
                                                                .columns[::-1]].cumsum(axis=1)
segment_purchase_retention_raw = segment_purchase_retention_raw[
    segment_purchase_retention_raw.columns[::-1]]
segment_purchase_retention_raw["num_users"] = segment_purchase_retention_raw[0]
segment_purchase_retention_raw = segment_purchase_retention_raw.groupby(
    ['segment_name', 'segment_id']).sum()
segment_purchase_retention = pd.concat([segment_purchase_retention_raw["num_users"],
                                        segment_purchase_retention_raw
                                        .drop(["num_users"], axis=1)
                                        .div(segment_purchase_retention_raw["num_users"], axis=0)],
                                       axis=1).fillna(0)
segment_purchase_retention=segment_purchase_retention.drop(0,axis=1)

In [ ]:
segment_purchase_stats_by_engagement_temp = pd.merge(segment_purchase_data, segment_engagement_user_data,
                                                     how='left',
                                                     left_on=[
                                                         'braze_id', 'segment_id', 'segment_name'],
                                                     right_on=[
                                                         'braze_id', 'segment_id', 'segment_name'],
                                                     suffixes=['_from_retention', '_from_engagement'])

segment_purchase_stats_by_engagement_temp['day_num'] = (
    segment_purchase_stats_by_engagement_temp['last_purchase_date'] - segment_purchase_stats_by_engagement_temp['first_purchase_date']).dt.days
segment_purchase_stats_by_engagement_temp['channel'] = segment_purchase_stats_by_engagement_temp.channel.fillna(
    'No Messages')
segment_purchase_stats_by_engagement_temp['channel_combo'] = segment_purchase_stats_by_engagement_temp.channel_combo.fillna(
    'No Messages')

In [ ]:
segment_purchase_stats_by_channel_temp = segment_purchase_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel']).agg(buyers=('braze_id', 'nunique'), repeat_buyers=('repeat_buyer_id', 'nunique'), num_purchases=(
    'num_purchases', 'sum'), total_revenue=('total_revenue', 'sum'), avg_revenue_per_buyer=('total_revenue', 'mean'), avg_time_to_purchase=('days_to_purchase', 'mean'), total_buyers=('braze_id', 'count'), total_repeat_buyers=('repeat_buyer_id', 'count'))
segment_purchase_stats_by_channel_temp['pct_repeat_buyers'] = round(
    segment_purchase_stats_by_channel_temp.repeat_buyers/segment_purchase_stats_by_channel_temp.buyers, 2)
segment_purchase_stats_by_channel_temp['purchases_per_buyer'] = round(
    segment_purchase_stats_by_channel_temp.num_purchases/segment_purchase_stats_by_channel_temp.total_buyers, 2)
segment_purchase_stats_by_channel_temp['revenue_per_item_purchased'] = round(
    segment_purchase_stats_by_channel_temp.total_revenue/segment_purchase_stats_by_channel_temp.num_purchases, 2)
segment_purchase_stats_by_channel = pd.merge(segment_purchase_stats_by_channel_temp, users_per_channel_df,
                                             how='left',
                                             left_on=[
                                                 'segment_name', 'segment_id','channel'],
                                             right_on=['segment_name', 'segment_id','channel'])

segment_purchase_stats_by_channel['pct_buyers'] = round(
    segment_purchase_stats_by_channel.buyers/segment_purchase_stats_by_channel.num_users, 2)

segment_purchase_stats_by_channel = segment_purchase_stats_by_channel[['segment_name', 'segment_id', 'channel', 'buyers', 'repeat_buyers', 'num_users', 'pct_buyers',
                                                                       'pct_repeat_buyers', 'purchases_per_buyer', 'avg_revenue_per_buyer', 'avg_time_to_purchase', 'revenue_per_item_purchased']].set_index(['segment_name', 'segment_id', 'channel'])


In [ ]:
segment_purchase_stats_by_channel_combo_temp = segment_purchase_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel_combo']).agg(buyers=('braze_id', 'nunique'), repeat_buyers=('repeat_buyer_id', 'nunique'), num_purchases=(
    'num_purchases', 'sum'), total_revenue=('total_revenue', 'sum'), avg_revenue_per_buyer=('total_revenue', 'mean'), avg_time_to_purchase=('days_to_purchase', 'mean'), total_buyers=('braze_id', 'count'), total_repeat_buyers=('repeat_buyer_id', 'count'))

segment_purchase_stats_by_channel_combo_temp['pct_repeat_buyers'] = round(
    segment_purchase_stats_by_channel_combo_temp.repeat_buyers/segment_purchase_stats_by_channel_combo_temp.buyers, 2)
segment_purchase_stats_by_channel_combo_temp['purchases_per_buyer'] = round(
    segment_purchase_stats_by_channel_combo_temp.num_purchases/segment_purchase_stats_by_channel_combo_temp.total_buyers, 2)
segment_purchase_stats_by_channel_combo_temp['revenue_per_item_purchased'] = round(
    segment_purchase_stats_by_channel_combo_temp.total_revenue/segment_purchase_stats_by_channel_combo_temp.num_purchases, 2)
segment_purchase_stats_by_channel_combo = pd.merge(segment_purchase_stats_by_channel_combo_temp, users_per_channel_combo_df,
                                                   how='left',
                                                   left_on=[
                                                       'segment_name', 'segment_id','channel_combo'],
                                                   right_on=['segment_name', 'segment_id','channel_combo'])

segment_purchase_stats_by_channel_combo['pct_buyers'] = round(
    segment_purchase_stats_by_channel_combo.buyers/segment_purchase_stats_by_channel_combo.num_users, 2)

segment_purchase_stats_by_channel_combo = segment_purchase_stats_by_channel_combo[['segment_name', 'segment_id', 'channel_combo', 'buyers', 'repeat_buyers', 'num_users', 'pct_buyers',
                                                                                   'pct_repeat_buyers', 'purchases_per_buyer', 'avg_revenue_per_buyer', 'avg_time_to_purchase', 'revenue_per_item_purchased']].set_index(['segment_name', 'segment_id', 'channel_combo'])

In [ ]:
segment_purchase_stats_by_campaign_tag_temp = segment_purchase_stats_by_engagement_temp.explode(
    'tags')
segment_purchase_stats_by_campaign_tag_temp = segment_purchase_stats_by_campaign_tag_temp.groupby(['segment_name', 'segment_id', 'tags']).agg(buyers=('braze_id', 'nunique'), repeat_buyers=('repeat_buyer_id', 'nunique'), num_purchases=(
    'num_purchases', 'sum'), total_revenue=('total_revenue', 'sum'), avg_revenue_per_buyer=('total_revenue', 'mean'), avg_time_to_purchase=('days_to_purchase', 'mean'), total_buyers=('braze_id', 'count'), total_repeat_buyers=('repeat_buyer_id', 'count'))
segment_purchase_stats_by_campaign_tag_temp['pct_repeat_buyers'] = round(
    segment_purchase_stats_by_campaign_tag_temp.repeat_buyers/segment_purchase_stats_by_campaign_tag_temp.repeat_buyers, 2)
segment_purchase_stats_by_campaign_tag_temp['purchases_per_buyer'] = round(
    segment_purchase_stats_by_campaign_tag_temp.num_purchases/segment_purchase_stats_by_campaign_tag_temp.total_buyers, 2)
segment_purchase_stats_by_campaign_tag_temp['revenue_per_item_purchased'] = round(
    segment_purchase_stats_by_campaign_tag_temp.total_revenue/segment_purchase_stats_by_campaign_tag_temp.num_purchases, 2)
segment_purchase_stats_by_campaign_tag = pd.merge(segment_purchase_stats_by_campaign_tag_temp, users_per_campaign_tags_df,
                                                  how='left',
                                                  left_on=[
                                                      'segment_name', 'tags'],
                                                  right_on=['segment_name', 'tags'])


segment_purchase_stats_by_campaign_tag['pct_buyers'] = round(
    segment_purchase_stats_by_campaign_tag.buyers/segment_purchase_stats_by_campaign_tag.num_users, 2)

segment_purchase_stats_by_campaign_tag = segment_purchase_stats_by_campaign_tag[['segment_name', 'segment_id', 'tags', 'buyers', 'repeat_buyers', 'num_users', 'pct_buyers',
                                                                                 'pct_repeat_buyers', 'purchases_per_buyer', 'avg_revenue_per_buyer', 'avg_time_to_purchase', 'revenue_per_item_purchased']].set_index(['segment_name', 'segment_id', 'tags'])

In [ ]:
segment_purchase_stats_by_engagement_temp['day_num'] = segment_purchase_stats_by_engagement_temp['day_num'].astype(
    'int')
segment_purchase_retention_by_channel_raw = pd.pivot_table(segment_purchase_stats_by_engagement_temp,
                                                           values=("braze_id"),
                                                           index=(
                                                               "segment_name", "segment_id", "channel"),
                                                           columns="day_num",
                                                           aggfunc='nunique')\
    .fillna(0)

segment_purchase_retention_by_channel_raw = segment_purchase_retention_by_channel_raw[segment_purchase_retention_by_channel_raw
                                                                                      .columns[::-1]].cumsum(axis=1)
segment_purchase_retention_by_channel_raw = segment_purchase_retention_by_channel_raw[
    segment_purchase_retention_by_channel_raw.columns[::-1]]
segment_purchase_retention_by_channel_raw["num_users"] = segment_purchase_retention_by_channel_raw[0]
segment_purchase_retention_by_channel_raw = segment_purchase_retention_by_channel_raw.groupby(
    ['segment_name', 'segment_id', "channel"]).sum()
segment_purchase_retention_by_channel = pd.concat([segment_purchase_retention_by_channel_raw["num_users"],
                                                   segment_purchase_retention_by_channel_raw
                                                   .drop(["num_users"], axis=1)
                                                   .div(segment_purchase_retention_by_channel_raw["num_users"], axis=0)],
                                                  axis=1).fillna(0)

segment_purchase_retention_by_channel=segment_purchase_retention_by_channel.drop(0, axis=1)


In [ ]:
segment_purchase_stats_by_engagement_temp['day_num'] = segment_purchase_stats_by_engagement_temp['day_num'].astype(
    'int')
segment_purchase_retention_by_channel_combo_raw = pd.pivot_table(segment_purchase_stats_by_engagement_temp,
                                                                 values=(
                                                                     "braze_id"),
                                                                 index=(
                                                                     "segment_name", "segment_id", "channel_combo"),
                                                                 columns="day_num",
                                                                 aggfunc='nunique')\
    .fillna(0)


segment_purchase_retention_by_channel_combo_raw = segment_purchase_retention_by_channel_combo_raw[segment_purchase_retention_by_channel_combo_raw
                                                                                                  .columns[::-1]].cumsum(axis=1)
segment_purchase_retention_by_channel_combo_raw = segment_purchase_retention_by_channel_combo_raw[
    segment_purchase_retention_by_channel_combo_raw.columns[::-1]]
segment_purchase_retention_by_channel_combo_raw[
    "num_users"] = segment_purchase_retention_by_channel_combo_raw[0]
segment_purchase_retention_by_channel_combo_raw = segment_purchase_retention_by_channel_combo_raw.groupby(
    ['segment_name', 'segment_id', "channel_combo"]).sum()
segment_purchase_retention_by_channel_combo = pd.concat([segment_purchase_retention_by_channel_combo_raw["num_users"],
                                                         segment_purchase_retention_by_channel_combo_raw
                                                         .drop(["num_users"], axis=1)
                                                         .div(segment_purchase_retention_by_channel_combo_raw["num_users"], axis=0)],
                                                        axis=1).fillna(0)

segment_purchase_retention_by_channel_combo=segment_purchase_retention_by_channel_combo.drop(0,axis=1)


In [ ]:
segment_purchase_stats_by_engagement_temp['day_num'] = segment_purchase_stats_by_engagement_temp['day_num'].astype(
    'int')
segment_purchase_stats_by_campaign_tag_temp = segment_purchase_stats_by_engagement_temp.explode(
    'tags')
segment_purchase_retention_by_campaign_tags_raw = pd.pivot_table(segment_purchase_stats_by_campaign_tag_temp,
                                                                 values=(
                                                                     "braze_id"),
                                                                 index=(
                                                                     "segment_name", "segment_id", "tags"),
                                                                 columns="day_num",
                                                                 aggfunc='nunique')\
    .fillna(0)

### Get the cumulative sum of users based on "last day"
segment_purchase_retention_by_campaign_tags_raw = segment_purchase_retention_by_campaign_tags_raw[segment_purchase_retention_by_campaign_tags_raw
                                                                                                  .columns[::-1]].cumsum(axis=1)
segment_purchase_retention_by_campaign_tags_raw = segment_purchase_retention_by_campaign_tags_raw[
    segment_purchase_retention_by_campaign_tags_raw.columns[::-1]]
segment_purchase_retention_by_campaign_tags_raw[
    "num_users"] = segment_purchase_retention_by_campaign_tags_raw[0]
segment_purchase_retention_by_campaign_tags_raw = segment_purchase_retention_by_campaign_tags_raw.groupby(
    ['segment_name', 'segment_id', "tags"]).sum()
segment_purchase_retention_by_campaign_tags = pd.concat([segment_purchase_retention_by_campaign_tags_raw["num_users"],
                                                         segment_purchase_retention_by_campaign_tags_raw
                                                         .drop(["num_users"], axis=1)
                                                         .div(segment_purchase_retention_by_campaign_tags_raw["num_users"], axis=0)],
                                                        axis=1).fillna(0)

segment_purchase_retention_by_campaign_tags=segment_purchase_retention_by_campaign_tags.drop(0,axis=1)


# Session Stats

The following Session Engagement Metrics will be returned by the codeblocks below:

- Number of Users
- Sessions per User

Session Engagement Metrics will returned by:

- Segment

- Channel

- Channel Combo

- Campaign Tag

In [ ]:
segment_sessions_stats_temp = segment_sessions_stats_pre_apps.explode('apps')
segment_sessions_stats_temp = segment_sessions_stats_temp.dropna(subset=[
                                                                 'apps'])
segment_sessions_stats_temp['sessions'] = segment_sessions_stats_temp['apps'].apply(
    lambda x: x.get('sessions'))
segment_sessions_stats_temp['platform'] = segment_sessions_stats_temp['apps'].apply(
    lambda x: x.get('platform'))

In [ ]:
segment_sessions_stats_temp = segment_sessions_stats_temp.groupby(['segment_name', 'segment_id']).agg(
    num_users=("braze_id", 'nunique'), total_sessions=('sessions', 'sum'))
segment_sessions_stats_temp['sessions_per_user'] = segment_sessions_stats_temp.total_sessions / \
    segment_sessions_stats_temp.num_users

segment_sessions_stats = segment_sessions_stats_temp

In [ ]:
segment_sessions_stats_by_engagement_temp = segment_engagement_user_data.explode(
    'apps')
segment_sessions_stats_by_engagement_temp = segment_sessions_stats_by_engagement_temp.dropna(subset=[
                                                                                             'apps'])
segment_sessions_stats_by_engagement_temp['sessions'] = segment_sessions_stats_by_engagement_temp['apps'].apply(
    lambda x: x.get('sessions'))
segment_sessions_stats_by_engagement_temp['platform'] = segment_sessions_stats_by_engagement_temp['apps'].apply(
    lambda x: x.get('platform'))

segment_sessions_stats_by_channel_temp = segment_sessions_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel']).agg(
    total_users=("braze_id", 'count'), total_sessions=('sessions', 'sum'), num_users=("braze_id", 'nunique'))
segment_sessions_stats_by_channel_temp = segment_sessions_stats_by_channel_temp.reset_index()
segment_sessions_stats_by_channel_temp['sessions_per_user'] = segment_sessions_stats_by_channel_temp.total_sessions / \
    segment_sessions_stats_by_channel_temp.total_users

segment_sessions_stats_by_channel = segment_sessions_stats_by_channel_temp[[
    'segment_name', 'segment_id', 'channel', 'num_users', 'sessions_per_user']].set_index(['segment_name', 'segment_id', 'channel'])

In [ ]:
segment_sessions_stats_by_channel_combo_temp = segment_sessions_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel_combo']).agg(
    total_users=("braze_id", 'count'), total_sessions=('sessions', 'sum'), num_users=("braze_id", 'nunique'))
segment_sessions_stats_by_channel_combo_temp = segment_sessions_stats_by_channel_combo_temp.reset_index()
segment_sessions_stats_by_channel_combo_temp['sessions_per_user'] = segment_sessions_stats_by_channel_combo_temp.total_sessions / \
    segment_sessions_stats_by_channel_combo_temp.total_users

segment_sessions_stats_by_channel_combo = segment_sessions_stats_by_channel_combo_temp[[
    'segment_name', 'segment_id', 'channel_combo', 'num_users', 'sessions_per_user']].set_index(['segment_name', 'segment_id', 'channel_combo'])

In [ ]:
segment_sessions_stats_by_campaign_tag_temp = segment_sessions_stats_by_engagement_temp.explode(
    'tags')
segment_sessions_stats_by_campaign_tag_temp = segment_sessions_stats_by_campaign_tag_temp.groupby(['segment_name', 'segment_id', 'tags']).agg(
    total_users=("braze_id", 'count'), total_sessions=('sessions', 'sum'), num_users=("braze_id", 'nunique'))
segment_sessions_stats_by_campaign_tag_temp = segment_sessions_stats_by_campaign_tag_temp.reset_index()
segment_sessions_stats_by_campaign_tag_temp['sessions_per_user'] = segment_sessions_stats_by_campaign_tag_temp.total_sessions / \
    segment_sessions_stats_by_campaign_tag_temp.total_users

segment_sessions_stats_by_campaign_tag = segment_sessions_stats_by_campaign_tag_temp[[
    'segment_name', 'segment_id', 'tags', 'num_users', 'sessions_per_user']].set_index(['segment_name', 'segment_id', 'tags'])

# Custom Event Stats

The following Custom Events Stats will be calculated:

- Number of Users Completing the Custom Event

- Number of Users

- Total Count of Custom Event

- % of Users Completing Custom Events

- Custom Events per User

- Avg. Days between each occurence of a Custom Event

- Avg. Custom Event Completion per Day

Custom Event stats dataframes will returned by:

- Segment

- Channel

- Channel Combo

- Campaign Tag

In [ ]:
segment_custom_event_stats_temp = segment_custom_event_stats_pre_custom_event.explode(
    'custom_events')

In [ ]:
segment_custom_event_stats_temp['custom_event_name'] = list(
    map(get_custom_event_name, segment_custom_event_stats_temp['custom_events']))
segment_custom_event_stats_temp['custom_event_count'] = list(
    map(get_custom_event_count, segment_custom_event_stats_temp['custom_events']))
segment_custom_event_stats_temp['custom_event_first_date'] = list(map(
    get_custom_event_first_date, segment_custom_event_stats_temp['custom_events']))
segment_custom_event_stats_temp['custom_event_last_date'] = list(
    map(get_custom_event_last_date, segment_custom_event_stats_temp['custom_events']))

In [ ]:
segment_custom_event_stats_temp[['custom_event_first_date', 'custom_event_last_date']] = segment_custom_event_stats_temp[[
    'custom_event_first_date', 'custom_event_last_date']].apply(pd.to_datetime)
segment_custom_event_stats_temp['days_between_events'] = (
    segment_custom_event_stats_temp['custom_event_last_date'] - segment_custom_event_stats_temp['custom_event_first_date']).dt.days
segment_custom_event_stats_temp['custom_event_per_day'] = np.round(np.where(segment_custom_event_stats_temp['days_between_events'] > 0,
                                                                   segment_custom_event_stats_temp.custom_event_count/segment_custom_event_stats_temp.days_between_events, segment_custom_event_stats_temp.custom_event_count), 1)

In [ ]:
total_segment_users_custom_event = segment_custom_event_stats_temp.braze_id.nunique()
segment_custom_event_stats_by_segment = segment_custom_event_stats_temp.groupby(
    ['segment_name', 'segment_id', 'custom_event_name']).agg(num_users_completing_custom_event=(
        'braze_id', 'nunique'), total_custom_events=('custom_event_count', 'sum'), avg_days_between_events=('days_between_events', 'mean'), avg_custom_event_per_day=('custom_event_per_day', 'mean'))
segment_custom_event_stats_by_segment['custom_event_per_user'] = segment_custom_event_stats_by_segment.total_custom_events / \
    total_segment_users_custom_event
segment_custom_event_stats_by_segment['pct_users_completing_custom_event'] = segment_custom_event_stats_by_segment.num_users_completing_custom_event / \
    total_segment_users_custom_event
segment_custom_event_stats_by_segment['num_users'] = total_segment_users_custom_event
segment_custom_event_stats_by_segment = segment_custom_event_stats_by_segment[[
    'num_users_completing_custom_event', 'num_users', 'total_custom_events', 'pct_users_completing_custom_event', 'custom_event_per_user', 'avg_days_between_events', 'avg_custom_event_per_day']]

In [ ]:
segment_custom_event_stats_by_engagement_temp = pd.merge(segment_custom_event_stats_temp, segment_engagement_user_data,
                                                         how='left',
                                                         left_on=[
                                                             'braze_id', 'segment_id', 'segment_name'],
                                                         right_on=[
                                                             'braze_id', 'segment_id', 'segment_name'],
                                                         suffixes=['_from_custom_events', '_from_engagement'])
segment_custom_event_stats_by_engagement_temp['channel'] = segment_custom_event_stats_by_engagement_temp.channel.fillna(
    'No Messages')
segment_custom_event_stats_by_engagement_temp['channel_combo'] = segment_custom_event_stats_by_engagement_temp.channel_combo.fillna(
    'No Messages')
segment_custom_event_stats_by_engagement_temp['tags'] = segment_custom_event_stats_by_engagement_temp.tags.fillna(
    'No Messages')

In [ ]:
segment_custom_event_stats_by_segment_and_channel_temp = segment_custom_event_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel', 'custom_event_name']).agg(num_users_completing_custom_event=(
    'braze_id', 'nunique'), total_custom_events=('custom_event_count', 'sum'), avg_days_between_events=('days_between_events', 'mean'), avg_custom_event_per_day=('custom_event_per_day', 'mean'))

segment_custom_event_stats_by_segment_and_channel_temp = segment_custom_event_stats_by_segment_and_channel_temp.reset_index()

segment_custom_event_stats_by_segment_and_channel = pd.merge(segment_custom_event_stats_by_segment_and_channel_temp, users_per_channel_df,
                                                             how='left',
                                                             left_on=[
                                                                 'segment_name', 'segment_id','channel'],
                                                             right_on=['segment_name', 'segment_id','channel'])
segment_custom_event_stats_by_segment_and_channel['custom_event_per_user'] = segment_custom_event_stats_by_segment_and_channel.total_custom_events / \
    segment_custom_event_stats_by_segment_and_channel.num_users
segment_custom_event_stats_by_segment_and_channel['pct_users_completing_custom_event'] = segment_custom_event_stats_by_segment_and_channel.num_users_completing_custom_event / \
    segment_custom_event_stats_by_segment_and_channel.num_users

segment_custom_event_stats_by_segment_and_channel = segment_custom_event_stats_by_segment_and_channel[['segment_name', 'segment_id', 'channel','custom_event_name', 'num_users_completing_custom_event', 'num_users',
                                                                                                       'total_custom_events', 'pct_users_completing_custom_event', 'custom_event_per_user', 'avg_days_between_events', 'avg_custom_event_per_day']].set_index(['segment_name', 'segment_id', 'channel'])


In [ ]:
segment_custom_event_stats_by_segment_and_channel_combo_temp = segment_custom_event_stats_by_engagement_temp.groupby(['segment_name', 'segment_id', 'channel_combo', 'custom_event_name']).agg(num_users_completing_custom_event=(
    'braze_id', 'nunique'), total_custom_events=('custom_event_count', 'sum'), avg_days_between_events=('days_between_events', 'mean'), avg_custom_event_per_day=('custom_event_per_day', 'mean'))

segment_custom_event_stats_by_segment_and_channel_combo_temp = segment_custom_event_stats_by_segment_and_channel_combo_temp.reset_index()
segment_custom_event_stats_by_segment_and_channel_combo = pd.merge(segment_custom_event_stats_by_segment_and_channel_combo_temp, users_per_channel_combo_df,
                                                                   how='left',
                                                                   left_on=[
                                                                       'segment_name', 'segment_id','channel_combo'],
                                                                   right_on=['segment_name', 'segment_id','channel_combo'])
segment_custom_event_stats_by_segment_and_channel_combo['custom_event_per_user'] = segment_custom_event_stats_by_segment_and_channel_combo.total_custom_events / \
    segment_custom_event_stats_by_segment_and_channel_combo.num_users
segment_custom_event_stats_by_segment_and_channel_combo['pct_users_completing_custom_event'] = segment_custom_event_stats_by_segment_and_channel_combo.num_users_completing_custom_event / \
    segment_custom_event_stats_by_segment_and_channel_combo.num_users

segment_custom_event_stats_by_segment_and_channel_combo = segment_custom_event_stats_by_segment_and_channel_combo[['segment_name', 'segment_id', 'channel_combo', 'custom_event_name','num_users_completing_custom_event', 'num_users',
                                                                                                                   'total_custom_events', 'pct_users_completing_custom_event', 'custom_event_per_user', 'avg_days_between_events', 'avg_custom_event_per_day']].set_index(['segment_name', 'segment_id', 'channel_combo'])


In [ ]:
segment_custom_event_stats_by_segment_and_campaign_tags_df = segment_custom_event_stats_by_engagement_temp.explode(
    'tags')
segment_custom_event_stats_by_segment_and_campaign_tags_temp = segment_custom_event_stats_by_segment_and_campaign_tags_df.groupby(['segment_name', 'segment_id', 'tags', 'custom_event_name']).agg(num_users_completing_custom_event=(
    'braze_id', 'nunique'), total_custom_events=('custom_event_count', 'sum'), avg_days_between_events=('days_between_events', 'mean'), avg_custom_event_per_day=('custom_event_per_day', 'mean'))

segment_custom_event_stats_by_segment_and_campaign_tags_temp = segment_custom_event_stats_by_segment_and_campaign_tags_temp.reset_index()
segment_custom_event_stats_by_segment_and_campaign_tags = pd.merge(segment_custom_event_stats_by_segment_and_campaign_tags_temp, users_per_campaign_tags_df,
                                                                   how='left',
                                                                   left_on=[
                                                                       'segment_name', 'segment_id','tags'],
                                                                   right_on=['segment_name', 'segment_id','tags'])
segment_custom_event_stats_by_segment_and_campaign_tags['custom_event_per_user'] = segment_custom_event_stats_by_segment_and_campaign_tags.total_custom_events / \
    segment_custom_event_stats_by_segment_and_campaign_tags.num_users
segment_custom_event_stats_by_segment_and_campaign_tags['pct_users_completing_custom_event'] = segment_custom_event_stats_by_segment_and_campaign_tags.num_users_completing_custom_event / \
    segment_custom_event_stats_by_segment_and_campaign_tags.num_users

segment_custom_event_stats_by_segment_and_campaign_tags = segment_custom_event_stats_by_segment_and_campaign_tags[[
    'segment_name', 'segment_id', 'tags', 'custom_event_name','num_users_completing_custom_event', 'num_users', 'total_custom_events', 'pct_users_completing_custom_event', 'custom_event_per_user', 'avg_days_between_events', 'avg_custom_event_per_day']].set_index(['segment_name', 'segment_id', 'tags'])


## Custom Events Used in Campaigns

The codeblock below will return all custom events that used in campaigns received by the selected segment.

In [ ]:
campaign_details_custom_event_temp = campaign_details_df[[
    'campaign_id', 'conversion_behaviors']]
campaign_details_custom_event_temp = campaign_details_custom_event_temp.dropna(
    subset=['conversion_behaviors'])
campaign_details_custom_event_temp = campaign_details_custom_event_temp.explode(
    'conversion_behaviors')

campaign_details_custom_event_temp['custom_event_conversion_behavior'] = list(map(
    get_campaign_custom_event, campaign_details_custom_event_temp['conversion_behaviors']))
campaign_details_custom_event_temp = campaign_details_custom_event_temp.dropna(
    subset=['custom_event_conversion_behavior'])
campaign_details_custom_event = campaign_details_custom_event_temp[[
    'campaign_id', 'custom_event_conversion_behavior']].drop_duplicates()
campaign_details_custom_event = campaign_details_custom_event.set_index(
    'campaign_id')

# Segment Opt-In Rates

The codeblock below will return the opt-in rates for Push and Email for all users across the following platforms:

- iOS
- Android
- Web

In [ ]:
segment_opt_ins_temp = segment_opt_in_stats_pre_apps.explode('apps')
segment_opt_ins_temp = segment_opt_ins_temp.dropna(subset=['apps'])
segment_opt_ins_temp = segment_opt_ins_temp.explode('push_tokens')

In [ ]:
segment_opt_ins_temp['notifications_enabled'] = list(
    map(get_notifications_enabled, segment_opt_ins_temp['push_tokens']))
segment_opt_ins_temp['token'] = list(
    map(get_token, segment_opt_ins_temp['push_tokens']))
segment_opt_ins_temp['push_token_platform'] = list(
    map(get_platform, segment_opt_ins_temp['push_tokens']))
segment_opt_ins_temp['app_platform'] = segment_opt_ins_temp['apps'].apply(
    lambda x: x.get('platform'))

In [ ]:
segment_opt_ins_temp_android = segment_opt_ins_temp[segment_opt_ins_temp['app_platform'] == 'Android'].copy()
segment_opt_ins_temp_android['push_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                           if x['notifications_enabled'] == True and x['token'] != "None"  else False, axis=1)
segment_opt_ins_temp_android['email_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                            if x['email_subscribe'] == 'opted_in' else False, axis=1)


segment_opt_ins_temp_ios = segment_opt_ins_temp[segment_opt_ins_temp['app_platform'] == 'iOS'].copy()
segment_opt_ins_temp_ios['push_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                       if x['notifications_enabled'] == True and x['token'] != "None" else False, axis=1)
segment_opt_ins_temp_ios['email_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                        if x['email_subscribe'] == 'opted_in' else False, axis=1)

segment_opt_ins_temp_web = segment_opt_ins_temp[segment_opt_ins_temp['app_platform'] == 'Web'].copy()
segment_opt_ins_temp_web['push_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                       if x['notifications_enabled'] == True and x['token'] != "None" else False, axis=1)
segment_opt_ins_temp_web['email_opted_in'] = segment_opt_ins_temp.apply(lambda x: True
                                                                        if x['email_subscribe'] == 'opted_in' else False, axis=1)

In [ ]:
segment_opt_ins_android_pre_agg = segment_opt_ins_temp_android.groupby(
    ['segment_id', 'segment_name', 'app_platform'])
opt_ins_aggregator = {'push_opted_in': calc_engagement,
                      'email_opted_in': calc_engagement}
segment_opt_ins_android = segment_opt_ins_android_pre_agg.agg(
    opt_ins_aggregator)
segment_opt_ins_ios_pre_agg = segment_opt_ins_temp_ios.groupby(
    ['segment_id', 'segment_name', 'app_platform'])
segment_opt_ins_ios = segment_opt_ins_ios_pre_agg.agg(opt_ins_aggregator)
segment_opt_ins_web_pre_agg = segment_opt_ins_temp_web.groupby(
    ['segment_id', 'segment_name', 'app_platform'])
segment_opt_ins_web = segment_opt_ins_web_pre_agg.agg(opt_ins_aggregator)
segment_opt_ins = pd.concat(
    [segment_opt_ins_android, segment_opt_ins_ios, segment_opt_ins_web])

## Exporting Outputs to Excel

Please note that attempting to export dataframes that were not created will result in an error.

In [ ]:
file_name = "Segment Analytics {date}.xlsx".format(date = datetime.now().date())
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

engagement_by_segment.to_excel(writer, sheet_name='Eng. by Segment')
engagement_by_segment_and_channel.to_excel(
    writer, sheet_name='Eng. by Channel')
engagement_by_segment_and_channel_combo.to_excel(
    writer, sheet_name='Eng. by Channel Combo')
engagement_by_segment_and_campaign_tag.to_excel(
    writer, sheet_name='Eng. by Campaign Tag')

segment_rolling_retention.to_excel(writer, sheet_name='Ret. by Segment')
segment_rolling_retention_by_engagement.to_excel(
    writer, sheet_name='Ret. by Channel')
segment_rolling_retention_by_channel_combo.to_excel(
    writer, sheet_name='Ret. by Channel Combo')
segment_rolling_retention_by_campaign_tag.to_excel(
    writer, sheet_name='Ret. by Campaign Tag')


segment_purchasing_stats.to_excel(writer, sheet_name='Purch. Stats by Segment')
segment_purchase_stats_by_channel.to_excel(
    writer, sheet_name='Purch. Stats by Channel')
segment_purchase_stats_by_channel_combo.to_excel(
    writer, sheet_name='Purch. Stats by Combo')
segment_purchase_stats_by_campaign_tag.to_excel(
    writer, sheet_name='Purch. Stats by Campaign Tag')
segment_purchase_retention.to_excel(writer, sheet_name='Purch. Ret by Segment')
segment_purchase_retention_by_channel.to_excel(
    writer, sheet_name='Purch. Ret by Channel')
segment_purchase_retention_by_channel_combo.to_excel(
    writer, sheet_name='Purch. Ret by Combo')
segment_purchase_retention_by_campaign_tags.to_excel(
    writer, sheet_name='Purch. Ret by Campaign Tag')

segment_sessions_stats.to_excel(writer, sheet_name='Sess. Stats by Segment')
segment_sessions_stats_by_channel.to_excel(
    writer, sheet_name='Sess. Stats by Channel')
segment_sessions_stats_by_channel_combo.to_excel(
    writer, sheet_name='Sess. Stats by Combo')
segment_sessions_stats_by_campaign_tag.to_excel(
    writer, sheet_name='Sess. Stats by Campaign Tag')

segment_custom_event_stats_by_segment.to_excel(
    writer, sheet_name='CE Stats by Segment')
segment_custom_event_stats_by_segment_and_channel.to_excel(
    writer, sheet_name='CE Stats by Channel')
segment_custom_event_stats_by_segment_and_channel_combo.to_excel(
    writer, sheet_name='CE Stats by Combo')
segment_custom_event_stats_by_segment_and_campaign_tags.to_excel(
    writer, sheet_name='CE Stats by Campaign Tag')
campaign_details_custom_event.to_excel(
    writer, sheet_name='CE Used in Campaigns')

segment_opt_ins.to_excel(writer, sheet_name='Opt-Ins by Segment')

writer.save()